I am exploring the data that was posted in the paper [Increases in COVID-19 are unrelated to levels of vaccination across 68 countries and 2947 counties in the United States](https://dx.doi.org/10.1007%2Fs10654-021-00808-7) it provides some plots showing the relationship between the percent of population vaccinated and the cases if COVID-19 infections. They conclude that the data show that there is no negative relationship between vaccination rates and infection rates.

This strikes me as counter to most of the research that shows a vaccine efficacy. So, here I pull the same data to replicate the results and perhaps explore further.

The paper reports getting their data from the Our World in Data database for the cross-country data. They take the data for the 7 days preceeding September 3 (when they pulled the data). I pulled [this version of the data](https://github.com/owid/covid-19-data/blob/b0bde8fa460b94938c47295880130a844e281539/public/data/owid-covid-data.xlsx) on October 30, 2021.

In [1]:
import yaml
import numpy
import pandas
import toyplot.svg

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     5.4.1
numpy version:    1.21.2
pandas version:   1.3.3
toyplot version:  0.19.0


In [2]:
full_data = pandas.read_excel('owid-covid-data.xlsx')

This data contains samples from Feburary 2020 to October 29 2021. We need to narrow the data to the range from August 28 to September 3 (the range used in the paper).

We going to drop any entry that has no continent information (suggesting that the entry is for a continent, not a country). We are also dropping entries that have not case information or population information as this is what was done in the paper. (It was probably invalid information anyway.)

In [3]:
full_data['datestamp'] = pandas.to_datetime(full_data['date'])
narrow_data = full_data[
    (full_data['datestamp'] >= pandas.Timestamp('2021-08-28')) &
    (full_data['datestamp'] <= pandas.Timestamp('2021-09-03'))]
narrow_data = narrow_data.dropna(
    subset=['continent', 'new_cases_per_million', 'population'], 
    how='any')

Sum up the case counts for all data within this range.

In [4]:
case_summation = pandas.pivot_table(
    narrow_data,
    index='location',
    values=[
        'new_cases_per_million',
        'new_tests_per_thousand',
    ],
    aggfunc='sum',
)
case_summation

,new_cases_per_million,new_tests_per_thousand
location,,
Afghanistan,10.418,0.000
Albania,2068.617,0.000
Algeria,69.973,0.000
Andorra,387.828,0.000
Angola,39.253,0.000
...,...,...
Venezuela,265.564,0.000
Vietnam,929.858,12.643
Yemen,10.069,0.000


In [5]:
case_summation.loc['Israel']

new_cases_per_million     8003.389
new_tests_per_thousand     108.473
Name: Israel, dtype: float64

A couple of observations here. First, we have over triple the values in this table as were reported in the paper. This is because (I believe) the paper removed any entries that did not have an update 3 days prior to September 3 when pulled on September 3. However, since then there have clearly been many updates to the tables for those countries that are slower at reporting. As a double-check, here is a quick check to show that countries have an entry on September 3.

In [6]:
date_check = pandas.pivot_table(
    narrow_data,
    index='location',
    values=['datestamp'],
    aggfunc='max',
)
date_check[date_check['datestamp'] != pandas.Timestamp('2021-09-03')]

,datestamp
location,


This brings me to the other difference. If you look at the values that are available in the paper, the case rates I have are larger. I believe this is for the same reason. At September 3, the data was not yet updated for all the values in this range. In the 2 months from then to now, all the reporting has completed, and we have a fuller (and more accurate) account.

So now with fuller data, let's continue. Let's pull the vaccination rate. I don't see where the paper specified which vaccination rate was picked (at the beginning or end of the week). I also found that the vaccination rate was not always posted and is missing in this particular range for some countries. So, I'll take the max reported at any time prior to the start since vaccines given during the week would not have any effect on the infection rate.

In [7]:
vac_summary = pandas.pivot_table(
    full_data[(full_data['datestamp'] <= pandas.Timestamp('2021-08-28'))],
    index='location',
    values=[
        'people_fully_vaccinated_per_hundred',
    ],
    aggfunc='max',
)
vac_summary

,people_fully_vaccinated_per_hundred
location,
Afghanistan,1.08
Africa,2.66
Albania,21.42
Algeria,1.62
Andorra,52.51
...,...
Wallis and Futuna,43.06
World,26.63
Yemen,0.04


Once again, my numbers don't match up with the paper. The values in the paper are greater. I think they did some averaging. I don't know what the point of that is.

Anyway, let's combine the data. Some of the countries are missing vaccination information, so we also have to remove those.

In [8]:
plot_data = pandas.DataFrame({
    'cases_per_million': case_summation['new_cases_per_million'],
    'vac_percent': vac_summary['people_fully_vaccinated_per_hundred'],
    'tests_per_thousand': case_summation['new_tests_per_thousand'],
}).dropna()
plot_data

,cases_per_million,vac_percent,tests_per_thousand
location,,,
Afghanistan,10.418,1.08,0.000
Albania,2068.617,21.42,0.000
Algeria,69.973,1.62,0.000
Andorra,387.828,52.51,0.000
Angola,39.253,2.15,0.000
...,...,...,...
Venezuela,265.564,11.62,0.000
Vietnam,929.858,2.48,12.643
Yemen,10.069,0.04,0.000


Finally, let's recreate the plot.

In [9]:
plot_data[plot_data['cases_per_million'] > 7300]

,cases_per_million,vac_percent,tests_per_thousand
location,,,
Israel,8003.389,62.31,108.473
Mongolia,7403.998,62.60,7.395


In [10]:
canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Population Fully Vaccinated (%)',
    ylabel='COVID-19 Cases per Million People in 7 Days',
)
x = plot_data['vac_percent']
y = plot_data['cases_per_million']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

axes.scatterplot(x, y)

axes.text(
    plot_data['vac_percent']['Israel'],
    plot_data['cases_per_million']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t53100a4b6f3a412ca46b14020d81569c"> Israel 0 25 50 75 Population Fully Vaccinated (%) 0 2500 5000 7500 COVID-19 Cases per Million People in 7 Days

In [11]:
toyplot.svg.render(canvas, 'vaccinated-vs-cases.svg')

This looks pretty similar to the table given in the paper.

A hypothesis I have is that the reason you are getting fewer cases with low vaccination rates is simply that more infections are going unreported. A reason for that might be that fewer tests are being adminsistered. Out of curiosity, lets color the points by the number of tests administered. (Many are reported as 0, probably because those countries did not report the number of tests.)

In [12]:
canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Population Fully Vaccinated (%)',
    ylabel='COVID-19 Cases per Million People in 7 Days',
)
x = plot_data['vac_percent']
y = plot_data['cases_per_million']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

colors = numpy.log(numpy.array(plot_data['tests_per_thousand']) + 1)
colormap = toyplot.color.brewer.map(
    "BlueGreenYellow", domain_max=0, domain_min=numpy.max(colors))

axes.scatterplot(x, y, color=(colors, colormap))

axes.text(
    plot_data['vac_percent']['Israel'],
    plot_data['cases_per_million']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8ae3f31bcc7c413781f3aa69ac2c68e1"> Israel 0 25 50 75 Population Fully Vaccinated (%) 0 2500 5000 7500 COVID-19 Cases per Million People in 7 Days

Let's take a closer look at the relationship between the vaccinated population and the number of tests. To be fair, we will throw away all countries that do not report any tests.

In [13]:
plot_data = plot_data[plot_data['tests_per_thousand'] > 0]

In [14]:
canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Population Fully Vaccinated (%)',
    ylabel='Tests per Thousand People',
)
x = plot_data['vac_percent']
y = plot_data['tests_per_thousand']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

axes.scatterplot(x, y)

axes.text(
    plot_data['vac_percent']['Israel'],
    plot_data['tests_per_thousand']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfdc86ee124dc475394c4872bc0ef3ef3"> Israel 0 25 50 75 Population Fully Vaccinated (%) 0 100 200 300 400 Tests per Thousand People

In [15]:
toyplot.svg.render(canvas, 'vaccinated-vs-tests.svg')

So, we see a positive correlation between the number of vaccinations and the number of tests. That is not surprising. A country with the wherewithal and desire to vaccinate a high proportion of people are also those that are more willing and able to adminster more tests.

It would also make sense if the number of tests affected the number of positive cases. After all, a person cannot test positive if that person is not tested in the first place. Let's check that hypothesis to see if, ignoring the vaccination rate, more tests is correlated with more _reported_ infections.

In [16]:
canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Tests per Thousand People',
    ylabel='COVID-19 Cases per Million People in 7 Days',
)
x = plot_data['tests_per_thousand']
y = plot_data['cases_per_million']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

axes.scatterplot(x, y)

axes.text(
    plot_data['tests_per_thousand']['Israel'],
    plot_data['cases_per_million']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te5022c996d604618a5934b9f7ed0b944"> Israel 0 100 200 300 400 Tests per Thousand People 0 2500 5000 7500 COVID-19 Cases per Million People in 7 Days

In [17]:
toyplot.svg.render(canvas, 'tests-vs-cases.svg')

Yes, there is a correlation here, too. No surprise here.

So, to return to the original conclusion of the paper, it is wrong to say that "there appears to be no discernable relationship between percentage of population fully vaccinated and new COVID-19 cases." The key here is that the data are the number of _reported_ cases, not the number of _actual_ cases. What the data are actually showing is the obvious correlations of vaccination to the number of tests and number of tests to infections discovered.

But let's not stop there. Let's try to dive a little deeper. Rather than look at the number of cases _reported_, let's look at the number of deaths attributed to COVID-19. A dealth is more likely to be investigated and reported than someone who is not seriously ill. Granted, some nations will be more vigilent than others, but an established health care system should catch most of them. This is probably why the [CDC uses hospitalizations and deaths as an indicator of vaccination efficacy](https://covid.cdc.gov/covid-data-tracker/#vaccine-effectiveness) rather than infection.

Let's repeat this experiment using deaths instead of infections.

In [18]:
case_summation = pandas.pivot_table(
    narrow_data,
    index='location',
    values=[
        'new_cases_per_million',
        'new_tests_per_thousand',
        'new_deaths_per_million',
    ],
    aggfunc='sum',
)
case_summation

,new_cases_per_million,new_deaths_per_million,new_tests_per_thousand
location,,,
Afghanistan,10.418,0.653,0.000
Albania,2068.617,7.308,0.000
Algeria,69.973,5.043,0.000
Andorra,387.828,0.000,0.000
Angola,39.253,1.827,0.000
...,...,...,...
Venezuela,265.564,3.518,0.000
Vietnam,929.858,24.376,12.643
Yemen,10.069,2.164,0.000


In [19]:
plot_data = pandas.DataFrame({
    'cases_per_million': case_summation['new_cases_per_million'],
    'deaths_per_million': case_summation['new_deaths_per_million'],
    'vac_percent': vac_summary['people_fully_vaccinated_per_hundred'],
    'tests_per_thousand': case_summation['new_tests_per_thousand'],
}).dropna()
plot_data

,cases_per_million,deaths_per_million,vac_percent,tests_per_thousand
location,,,,
Afghanistan,10.418,0.653,1.08,0.000
Albania,2068.617,7.308,21.42,0.000
Algeria,69.973,5.043,1.62,0.000
Andorra,387.828,0.000,52.51,0.000
Angola,39.253,1.827,2.15,0.000
...,...,...,...,...
Venezuela,265.564,3.518,11.62,0.000
Vietnam,929.858,24.376,2.48,12.643
Yemen,10.069,2.164,0.04,0.000


In [20]:
canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Population Fully Vaccinated (%)',
    ylabel='COVID-19 Deaths per Million People in 7 Days',
)
x = plot_data['vac_percent']
y = plot_data['deaths_per_million']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

axes.scatterplot(x, y)

axes.text(
    plot_data['vac_percent']['Israel'],
    plot_data['deaths_per_million']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taa2dda8a265e4a318dd573c86d3d78e5"> Israel 0 25 50 75 Population Fully Vaccinated (%) 0 50 100 150 COVID-19 Deaths per Million People in 7 Days

This looks different than the comparison of reported infections. In particular, where before Israel was an anomaly (high vaccination high infections), it is now closer to the norm (high vaccination low deaths). The correlation is ever so slightly down, but not enough to really draw conclusions.

But notice that clump in the lower left of the graph. There are lots of countries with low population rates and reporting low death rates. Once again, we need to remind our self that we are dealing with the number of COVID-19 deaths _reported_, not those that actually happened. Although most "first world countries" are likely to check and report all deaths that are likely caused by COVID, a country with a very poor healthcare infrastructure is likely to be unable to investigate causes of death.

If we look closer, we see many rows in the data that report 0 deaths, many of which also have a high infection rate.

In [21]:
pandas.set_option('display.max_rows', None)
plot_data[plot_data['deaths_per_million'] == 0]

,cases_per_million,deaths_per_million,vac_percent,tests_per_thousand
location,,,,
Andorra,387.828,0.0,52.51,0.000
Bahrain,376.940,0.0,62.06,79.107
Bhutan,5.128,0.0,60.96,0.000
Chad,0.354,0.0,0.09,0.000
China,0.135,0.0,61.59,0.000
Comoros,31.516,0.0,4.66,0.000
Congo,9.722,0.0,1.86,0.000
Djibouti,46.897,0.0,2.28,0.000
Hong Kong,2.383,0.0,44.15,0.000


many of the reports of 0 death seem suspicious. How do we correct for that? Well, we could just remove them. But what about countries like Liechtenstein and Luxembourg? These countries should be able to accurately report deaths, but are probably so small that there just happened to be no deaths that week. We don't want to skew our data for these cases.

However, what if we look at the number of hospital beds per capita? Shouldn't that be a good indication of the availability of healthcare (and therefore the likelyhood of a COVID-19 death being detected). What happens if we limit the plot to countries that have more than 1 bed for every thousand people.

In [22]:
vac_summary = pandas.pivot_table(
    full_data[(full_data['datestamp'] <= pandas.Timestamp('2021-08-28'))],
    index='location',
    values=[
        'people_fully_vaccinated_per_hundred',
        'hospital_beds_per_thousand',
    ],
    aggfunc='max',
).dropna()
vac_summary = vac_summary[vac_summary['hospital_beds_per_thousand'] > 1]

In [23]:
pandas.set_option('display.max_rows', 10)
plot_data = pandas.DataFrame({
    'cases_per_million': case_summation['new_cases_per_million'],
    'deaths_per_million': case_summation['new_deaths_per_million'],
    'vac_percent': vac_summary['people_fully_vaccinated_per_hundred'],
    'tests_per_thousand': case_summation['new_tests_per_thousand'],
    'beds': vac_summary['hospital_beds_per_thousand'],
}).dropna()
plot_data

,cases_per_million,deaths_per_million,vac_percent,tests_per_thousand,beds
location,,,,,
Albania,2068.617,7.308,21.42,0.000,2.89
Algeria,69.973,5.043,1.62,0.000,1.90
Antigua and Barbuda,1539.584,10.129,33.82,0.000,3.80
Argentina,705.743,23.813,29.87,11.980,5.00
Armenia,1253.315,32.344,2.88,11.002,4.20
...,...,...,...,...,...
Uruguay,255.942,2.296,71.63,13.565,2.80
Uzbekistan,157.858,1.267,3.63,0.000,4.00
Vietnam,929.858,24.376,2.48,12.643,2.60


In [24]:
canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Population Fully Vaccinated (%)',
    ylabel='COVID-19 Deaths per Million People in 7 Days',
)
x = plot_data['vac_percent']
y = plot_data['deaths_per_million']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

axes.scatterplot(x, y)

axes.text(
    plot_data['vac_percent']['Israel'],
    plot_data['deaths_per_million']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3c7a3c570dcb4323b86ecff2e7a3ad83"> Israel 0 25 50 75 Population Fully Vaccinated (%) 0 50 100 150 COVID-19 Deaths per Million People in 7 Days

Although there still is a clump of countries in the lower left corner that is probably under-reporting deaths, we have managed to filter many of them out. Here we see that yes in fact there is a negative correlation between COVID-19 deaths and vaccinations.

I noticed that the dataset also has some information about excess mortality. Although this metric is not perfect, it is a bit less dependent on accurately determining whether or not deaths are caused by COVID-19.

In [25]:
vac_summary = pandas.pivot_table(
    full_data[(full_data['datestamp'] <= pandas.Timestamp('2021-08-28'))],
    index='location',
    values=[
        'people_fully_vaccinated_per_hundred',
        'hospital_beds_per_thousand',
        'excess_mortality',
    ],
    aggfunc='max',
).dropna()

plot_data = pandas.DataFrame({
    'cases_per_million': case_summation['new_cases_per_million'],
    'deaths_per_million': case_summation['new_deaths_per_million'],
    'vac_percent': vac_summary['people_fully_vaccinated_per_hundred'],
    'tests_per_thousand': case_summation['new_tests_per_thousand'],
    'beds': vac_summary['hospital_beds_per_thousand'],
    'excess_mortality': vac_summary['excess_mortality'],
}).dropna()

canvas = toyplot.Canvas(width='400px', height='400px')
axes = canvas.cartesian(
    xlabel='Population Fully Vaccinated (%)',
    ylabel='Excess Mortality (%)',
)
x = plot_data['vac_percent']
y = plot_data['excess_mortality']

fit_coef = numpy.polyfit(x, y, 1)

fit_x = numpy.array([numpy.min(x), numpy.max(x)])
fit_y = fit_x*fit_coef[0] + fit_coef[1]
axes.plot(fit_x, fit_y, color='#BBBBBB')

axes.scatterplot(x, y)

axes.text(
    plot_data['vac_percent']['Israel'],
    plot_data['excess_mortality']['Israel'],
    'Israel',
    style={'text-anchor':'start',
           '-toyplot-anchor-shift':'5pt'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5c6f42d2f4524814a9e2400d13765491"> Israel 0 25 50 75 Population Fully Vaccinated (%) 0 100 200 300 400 Excess Mortality (%)